In [1]:
import sys
# import gc  # debug that memory leak
# import tracemalloc  # DEBUG THAT MEMORY LEAK
import psutil  # :( mem leak
import h5py
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader
from  torch.nn.functional import one_hot
from sklearn.utils import shuffle
import torch
from torch import nn
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from datetime import datetime
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")
dtype = torch.double

# Get functions from other notebooks
%run /tigress/kendrab/analysis-notebooks/torch_models/utils.ipynb
%run /tigress/kendrab/analysis-notebooks/preproc_utils.ipynb
%run /tigress/kendrab/analysis-notebooks/torch_models/import_mms_data.ipynb

Using cpu device


In [2]:
# parameters
mms_epochs = 1  # number of times to loop through the entire mms dataset (start with 1 lmao)

In [3]:
# TODO : make notebook to load sim data x
# TODO : there is TOO MUCH MMS data to load it all at once. Need to figure out how to do the training in batches
# TODO : set up the ADDA architecture x
# TODO : see what it spits out / how to interpret the adaptation results (discriminator metric?)

In [4]:
# load a model
%run /tigress/kendrab/analysis-notebooks/torch_models/import_model.ipynb

# TODO?: enforce that key variables that need to exist later down the pipeline
# are populated by import_model?

/home/kendrab/.conda/envs/torch-env/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
# extract the model's features (before classification step)
# get_graph_node_names(model)
return_nodes = {'post_merge_layers.2' : 'features'}
feat_sim = create_feature_extractor(model, return_nodes=return_nodes)

In [6]:
# build the domain adaptation structure
%run /tigress/kendrab/analysis-notebooks/torch_models/adda_constructor.ipynb

Discriminator(
  (layer1): Sequential(
    (0): LazyLinear(in_features=0, out_features=60, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (layer2): Sequential(
    (0): Linear(in_features=60, out_features=60, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
  (domain_label): Sequential(
    (0): Linear(in_features=60, out_features=1, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
  )
)
MMSFeatExtract(
  (bx_layers): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): LeakyReLU(negative_slope=0.01)
    (2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (by_layers): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): LeakyReLU(negative_slope=0.01)
    (2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  )
  (bz_layers): Sequential(
    (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=valid)
    (1): LeakyReLU(negative_slope=0.01)
    (2): AvgPool1d(kernel_size=

In [7]:
# Load the sim data
%run /tigress/kendrab/analysis-notebooks/torch_models/import_sim_data.ipynb

(537972, 1, 30)
(537972, 2, 10)
torch.float64


In [8]:
# Load the mms data locations and shuffle the files to not be chronological
mms_filenames = get_filenames()
debug_filename = "2021-08-18T04-48-00_2021-08-20T04-30-30.h5"

### Time to do some training 

In [9]:
loss_fn = nn.BCEWithLogitsLoss()
optim_disc = torch.optim.Adam(discrim.parameters(),lr=learning_rate)
optim_feat = torch.optim.Adam(feat_mms.parameters(),lr=learning_rate)
loss_feat = []
loss_disc = []

In [10]:
for epoch in range(mms_epochs):
    print(f"Starting Epoch {epoch+1}")
    mms_filenames = shuffle(mms_filenames)
    sim_iter = iter(sim_dl)  # to make sure we loop through the whole dataset before starting over even as we switch between mms files
    for i, mms_file in enumerate(mms_filenames):
        print(f"getting MMS file {mms_file}, number {i+1} of {len(mms_filenames)}")
        # get the mms data from the file
        process = psutil.Process()  # start debug loop thingy
        print(process.memory_info().rss/1024/1024)
        mms_data_dict = get_mms_data(mms_file)
        print(process.memory_info().rss/1024/1024)
        if len(mms_data_dict) == 0:  # data was not loaded, skip this file
            continue
        mms_dl = format_mms_data(mms_data_dict)
        print(process.memory_info().rss/1024/1024)            
        training_step = train_adda(sim_dl, mms_dl, feat_sim, feat_mms, discrim, loss_fn, optim_disc,
                                   optim_feat, iter_source = sim_iter)
        sim_iter = training_step["iter_source"]
        loss_feat = training_step["loss_feat"]
        loss_disc = training_step["loss_disc"]

Starting Epoch 1
getting MMS file 2022-07-11T07-38-30_2022-07-11T23-28-00.h5, number 1 of 230
overridden with debug file 2021-08-18T04-48-00_2021-08-20T04-30-30.h5
2407.125
2659.25
2716.8125
getting MMS file 2018-10-03T23-00-30_2018-10-03T23-52-00.h5, number 2 of 230
overridden with debug file 2021-08-18T04-48-00_2021-08-20T04-30-30.h5
2633.125
2764.0


KeyboardInterrupt: 

In [ ]:
# plot the loss
fig, ax = plt.subplots(2)
ax[0].plot(loss_disc)
ax[1].plot(loss_feat)
ax[0].set(title="Discriminator loss", xlabel="training iteration", ylabel="loss")
ax[1].set(title="MMS Feature Extractor loss", xlabel="training iteration", ylabel="loss")
fig.savefig("/tigress/kendrab/analysis-notebooks/model_outs/scratchwork/training_losses.svg")  # TODO: save model and training info to its own folder
plt.close(fig='all')